In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [2]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)

In [3]:
basics_df =  basics_df.drop('endYear',axis=1)
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...
8442829,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,"Action,Drama,Family"
8442830,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,"Action,Drama,Family"
8442831,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,"Action,Drama,Family"
8442832,tt9916856,short,The Wind,The Wind,0,2015,27,Short


In [4]:
movies = basics_df.loc[basics_df['titleType']=='movie']

In [5]:
movies = movies.drop('titleType',axis=1)

In [6]:
movies

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
498,tt0000502,Bohemios,Bohemios,0,1905,100,\N
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama
610,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,\N,Drama
625,tt0000630,Hamlet,Amleto,0,1908,\N,Drama
...,...,...,...,...,...,...,...
8442723,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary
8442750,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary
8442762,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013,\N,Comedy
8442773,tt9916730,6 Gunn,6 Gunn,0,2017,116,\N


In [7]:
movies_ok = movies.loc[movies['isAdult']=='0']
movies_ok=movies_ok.drop('isAdult',axis=1)
movies_ok

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
498,tt0000502,Bohemios,Bohemios,1905,100,\N
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama
610,tt0000615,Robbery Under Arms,Robbery Under Arms,1907,\N,Drama
625,tt0000630,Hamlet,Amleto,1908,\N,Drama
...,...,...,...,...,...,...
8442723,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,57,Documentary
8442750,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,2007,100,Documentary
8442762,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,\N,Comedy
8442773,tt9916730,6 Gunn,6 Gunn,2017,116,\N


In [8]:
movies_ok_nat = movies_ok.replace('\\N', pd.NaT)
movies_clean = movies_ok_nat.dropna()

In [9]:
movies_clean

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama
672,tt0000679,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,1908,120,"Adventure,Fantasy"
1172,tt0001184,Don Juan de Serrallonga,Don Juan de Serrallonga,1910,58,"Adventure,Drama"
1246,tt0001258,The White Slave Trade,Den hvide slavehandel,1910,45,Drama
...,...,...,...,...,...,...
8442597,tt9916362,Coven,Akelarre,2020,92,"Drama,History"
8442682,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama
8442723,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,57,Documentary
8442750,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,2007,100,Documentary


In [10]:
movies_clean = movies_clean.astype({'runtimeMinutes': int})
movies_clean = movies_clean.astype({'startYear': int})

In [11]:
movies_clean = movies_clean[(movies_clean['runtimeMinutes'] >= 58) & (movies_clean['runtimeMinutes'] <= 270)]
movies_clean

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama
672,tt0000679,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,1908,120,"Adventure,Fantasy"
1172,tt0001184,Don Juan de Serrallonga,Don Juan de Serrallonga,1910,58,"Adventure,Drama"
1773,tt0001790,"Les Misérables, Part 1: Jean Valjean",Les misérables - Époque 1: Jean Valjean,1913,60,Drama
...,...,...,...,...,...,...
8442513,tt9916190,Safeguard,Safeguard,2020,90,"Action,Adventure,Thriller"
8442552,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,Thriller
8442597,tt9916362,Coven,Akelarre,2020,92,"Drama,History"
8442682,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama


In [12]:
movies_clean = movies_clean[(movies_clean['startYear'] >= 1918) & (movies_clean['startYear'] <= 2021)]
movies_clean

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
2396,tt0002423,Passion,Madame DuBarry,1919,85,"Biography,Drama,Romance"
5013,tt0005076,Charley's Aunt,Charley's Aunt,1925,80,Comedy
6979,tt0007068,Mind the Paint Girl,Mind the Paint Girl,1919,60,"Drama,Romance"
7328,tt0007422,Das Tagebuch des Dr. Hart,Das Tagebuch des Dr. Hart,1918,82,"Drama,Romance"
7449,tt0007547,Der Weg des Todes,Der Weg des Todes,1918,75,Drama
...,...,...,...,...,...,...
8442513,tt9916190,Safeguard,Safeguard,2020,90,"Action,Adventure,Thriller"
8442552,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,Thriller
8442597,tt9916362,Coven,Akelarre,2020,92,"Drama,History"
8442682,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama


In [13]:
movies_clean[['mainGenre', 'secondaryGenres','thirdGenre']] = movies_clean['genres'].str.split(',', expand=True)

C:\Users\josse\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
movies_clean

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,mainGenre,secondaryGenres,thirdGenre
2396,tt0002423,Passion,Madame DuBarry,1919,85,"Biography,Drama,Romance",Biography,Drama,Romance
5013,tt0005076,Charley's Aunt,Charley's Aunt,1925,80,Comedy,Comedy,None,None
6979,tt0007068,Mind the Paint Girl,Mind the Paint Girl,1919,60,"Drama,Romance",Drama,Romance,None
7328,tt0007422,Das Tagebuch des Dr. Hart,Das Tagebuch des Dr. Hart,1918,82,"Drama,Romance",Drama,Romance,None
7449,tt0007547,Der Weg des Todes,Der Weg des Todes,1918,75,Drama,Drama,None,None
...,...,...,...,...,...,...,...,...,...
8442513,tt9916190,Safeguard,Safeguard,2020,90,"Action,Adventure,Thriller",Action,Adventure,Thriller
8442552,tt9916270,Il talento del calabrone,Il talento del calabrone,2020,84,Thriller,Thriller,None,None
8442597,tt9916362,Coven,Akelarre,2020,92,"Drama,History",Drama,History,None
8442682,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,Drama,None,None


In [ ]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t")

In [93]:
movies = pd.merge(movies_clean,ratings_df,how='inner')

In [94]:
movies_7 = movies.loc[movies['averageRating']>=6]

In [95]:
movies_7_rating = movies_7[movies_7['numVotes'] >= 5000]


In [96]:
movies_7_rating = movies_7_rating.drop('genres',axis=1)

In [97]:
movies_7_rating

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,mainGenre,secondaryGenres,thirdGenre,averageRating,numVotes
91,tt0009968,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,1919,90,Drama,Romance,None,7.3,10025
132,tt0010323,The Cabinet of Dr. Caligari,Das Cabinet des Dr. Caligari,1920,76,Horror,Mystery,Thriller,8.1,60931
208,tt0011130,Dr. Jekyll and Mr. Hyde,Dr. Jekyll and Mr. Hyde,1920,69,Drama,Horror,Sci-Fi,7.0,5235
221,tt0011237,The Golem,"Der Golem, wie er in die Welt kam",1920,76,Fantasy,Horror,None,7.2,7379
296,tt0011841,Way Down East,Way Down East,1920,145,Drama,Romance,None,7.4,5370
...,...,...,...,...,...,...,...,...,...,...
222780,tt9866072,Holidate,Holidate,2020,104,Comedy,Romance,None,6.1,53916
222821,tt9877170,Malang,Malang - Unleash the Madness,2020,135,Action,Crime,Drama,6.5,9731
222879,tt9893250,I Care a Lot,I Care a Lot,2020,118,Comedy,Crime,Thriller,6.3,117218
222887,tt9894470,VFW,VFW,2019,92,Action,Crime,Horror,6.1,7486


In [21]:
crew_df = pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", sep="\t")

In [23]:
principals_df = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [24]:
name_df = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep="\t")

In [74]:
movies_dir = pd.merge(movies_7_rating,crew_df,how='inner')

In [75]:
movies_dir[['director1', 'director2']] = movies_dir['directors'].str.split(',', n=1, expand=True)

In [76]:
movies_dir.drop('directors',axis=1,inplace=True)
movies_dir.drop('writers',axis=1,inplace=True)

In [77]:
movies_dir.drop('tconst',axis=1,inplace=True)
movies_dir.drop('numVotes',axis=1,inplace=True)

In [78]:
movies_dir.drop('primaryTitle',axis=1,inplace=True)
movies_dir_rec = movies_dir.copy()

In [79]:
movies_dir_rec['mainGenre']  = movies_dir_rec['mainGenre'].factorize()[0]
movies_dir_rec['secondaryGenres']  = movies_dir_rec['secondaryGenres'].factorize()[0]
movies_dir_rec['thirdGenre']  = movies_dir_rec['thirdGenre'].factorize()[0]
movies_dir_rec['director1']  = movies_dir_rec['director1'].factorize()[0]
movies_dir_rec['director2']  = movies_dir_rec['director2'].factorize()[0]

In [80]:
movies_dir

,originalTitle,startYear,runtimeMinutes,mainGenre,secondaryGenres,thirdGenre,averageRating,director1,director2
0,Broken Blossoms or The Yellow Man and the Girl,1919,90,Drama,Romance,None,7.3,nm0000428,None
1,Das Cabinet des Dr. Caligari,1920,76,Horror,Mystery,Thriller,8.1,nm0927468,None
2,The Kid,1921,68,Comedy,Drama,Family,8.3,nm0000122,None
3,Körkarlen,1921,107,Drama,Fantasy,Horror,8.1,nm0803705,None
4,Häxan,1922,91,Documentary,Fantasy,History,7.7,nm0159725,None
...,...,...,...,...,...,...,...,...,...
4018,Nomadland,2020,107,Drama,None,None,7.4,nm2125482,None
4019,Judas and the Black Messiah,2021,126,Biography,Drama,History,7.5,nm3489851,None
4020,J'ai perdu mon corps,2019,81,Animation,Drama,Fantasy,7.6,nm3021346,None
4021,Metri Shesh Va Nim,2019,131,Action,Crime,Drama,8.0,nm7920254,None


In [81]:
X = movies_dir_rec.drop('originalTitle',axis=1)

In [82]:
distanceKNN = NearestNeighbors(n_neighbors=10).fit(X) 
suggestion =  distanceKNN.kneighbors([[2020,107,0,-1,-1,7.4,1847,-1]])
all_suggest = movies_dir.loc[suggestion[1][0]]
all_suggest

,originalTitle,startYear,runtimeMinutes,mainGenre,secondaryGenres,thirdGenre,averageRating,director1,director2
4018,Nomadland,2020,107,Drama,None,None,7.4,nm2125482,None
3803,Columbus,2017,104,Drama,None,None,7.3,nm3226379,None
3809,Professor Marston and the Wonder Women,2017,108,Biography,Drama,History,7.1,nm1286340,None
3861,Palmer,2021,110,Drama,None,None,7.3,nm0001770,None
3840,Newton,2017,106,Comedy,Drama,None,7.7,nm3536895,None
3839,Five Feet Apart,2019,116,Drama,Romance,None,7.2,nm1682573,None
3844,Salyut-7,2017,111,Action,Drama,History,7.2,nm2568708,None
3843,Fighting with My Family,2019,108,Biography,Comedy,Drama,7.1,nm0580351,None
3835,Downton Abbey,2019,122,Drama,Romance,None,7.4,nm0257554,None
3804,The Edge of Democracy,2019,121,Documentary,History,None,7.2,nm3525467,None
